# Precipitation in the BMI-ed Meteorology component

**Goal:** In this example, I want to give the `Meteorology` component a **time series** of precipitation values and check whether it produces output when the model state is updated.

Start with an import and some magic:

In [ ]:
%matplotlib inline
import numpy as np

Programmatically create a file holding the precipitation rate time series. This will mimic what I'll need to do in WMT, where I'll have access to the model time step and run duration. Start by defining the precipitation rate values:

In [ ]:
n_steps = 10  # can get from cfg file
precip_rates = np.linspace(5, 20, num=n_steps, endpoint=False)
precip_rates

Next, write the values to a file to the **input** directory, where it's expected by the cfg file: 

In [ ]:
np.savetxt('./input/precip_rates.txt', precip_rates, fmt='%6.2f')

Check the file:

In [ ]:
cat input/precip_rates.txt

Import the `Meteorology` component and create an instance:

In [ ]:
from topoflow.components.met_base import met_component
m = met_component()
m.initialize('./input/meteorology-1.cfg')

Unlike when `P` is a scalar, the initial model precipitation volume flux is the first value from **precip_rates.txt**, after converting units from mm/hr to m/s:

In [ ]:
precip = m.get_value('atmosphere_water__precipitation_leq-volume_flux')  # `P` internally
print type(precip)
print precip.size
precip

Advance the model by one time step:

In [ ]:
m.update()
print '\nCurrent time: {} s'.format(m.get_current_time())

Unlike the scalar case, there's an output volume flux of precipitation:

In [ ]:
print precip  # note that this is a reference, so it'll take the current value of `P`

Advance the model to the end, saving the model time and output `P` values (converted back to mm/hr for convenience) at each step:

In [ ]:
mps_to_mmph = 1000 * 3600
time = [m.get_current_time().copy()]
flux = [precip.copy() * mps_to_mmph]
while m.get_current_time() < m.get_end_time():
    m.update()
    time.append(m.get_current_time().copy())
    flux.append(m.get_value('atmosphere_water__precipitation_leq-volume_flux').copy() * mps_to_mmph)

Check the time and flux values:

In [ ]:
time

In [ ]:
flux

**Result:** Input precipipation rates match output precipitation volume flux. Behaves as expected!